# Machine Learning - Assessment 2

Through the following notebook, you will be tasked with **implementing** two different classification approaches: **k Nearest Neighbours** and **Decision Trees**. You will apply these to two different datasets, evaluate the produced models and analyse their performance.

This will be done through several different tasks and sub-tasks:
- [Dataset](#Dataset)
    - [Task 1: Dataset statistics **(10\%)**](#Task-1---Dataset-statistics)
- [k Nearest Neigbours](#k-Nearest-Neigbours)
    - [Task 2.1: Euclidean distance **(10\%)**](#Task-2.1---Euclidean-distance)
    - [Task 2.2: kNN classifier **(20\%)**](#Task-2.2---kNN-classifier)
- [Decision Trees](#Decision-Trees)
    - [Task 3.1: Entropy and Information Gain **(10\%)**](#Task-3.1---Entropy-and-Information-Gain)
    - [Task 3.2: Decision Tree classifier **(30\%)**](#Task-3.2---Decision-Tree-classifier)
- [Model evaluation and analysis](#Model-evaluation-and-analysis)
    - [Task 4.1: Evaluation **(10\%)**](#Task-4.1---Evaluation)
    - [Task 4.2: Analysis **(10\%)**](#Task-4.2---Analysis)

**Note:** The (%) noted above are out of 100; this will be scaled down to **maximum of 70 marks** for the assessment.

In [1]:
import math
import numpy as np
import pandas as pd
import collections

## Dataset

For this assessment, you will be working with two datasets:
- A [numeric-only](#Numerical-data) dataset (crystal systems of Li-ion batteries)
- A dataset with [mixed numeric and categorical](#Categorical-data) features (forest cover)

For developing your solutions, **you have only been provided with a portion of the samples from each of the dataset** (70% of the [numeric-only](#Numerical-data) and 50% of the [mixed numeric and categorical](#Categorical-data) data). These will be in **identical format** to the full datasets used for evaluating your solutions (there will just be more samples).

### Numerical data

The main part of the assessment will be done on the dataset containing only numerical features describing the physical and chemical properties of the Li-ion battery, which can be classified on the basis of their crystal system. Three major classes of crystal systems are: _monoclinic_, _orthorhombic_ and _triclinic_. (The dataset for this assessment has been adapted from the full dataset which can be found [here](https://www.kaggle.com/datasets/divyansh22/crystal-system-properties-for-liion-batteries

Each sample corresponds to the properties of a battery, and consists of following features:

| Feature Name      | Value | Description |
| :---------------- | :----- | ----------- |
| `Formation Energy`       | `float`: eV | Formation energy of the material. |
| `E Above Hull` | `float`: eV | Energy of decomposition of material into most stable ones. |
| `Band Gap` | `float`: eV | Band gap. |
| `Nsites` | `int`: count | Number of atoms in the unit cell of the crystal. |
| `Density` | `float`: gm/cc | The density of bulk crystalline materials. |
| `Volume` | `float` | The unit cell volume of the material. |

The goal for the assessment is to predict whether the crystal system of the battery is _monoclinic_, _orthorhombic_ or _triclinic_, which provides a classification for each sample:

| Class      | Value | Description |
| :---------------- | :----- | ----------- |
| `Crystal System`  | `string`: class designation | Class of the crystal system (three different values). |


This dataset will be used for the developmet and testing of both the [kNN](#k-Nearest-Neigbours) and [Decision Tree](#Decision-Trees) classifier, as well as for the [evaluation](#Task-4.1---Evaluation) of both of the classification approaches.

The following cell loads the dataset, and splits the samples randomly, using  $80\%$ of samples for training and $20\%$ for testing:

In [2]:
from sklearn.model_selection import train_test_split
from itertools import count

numerical_data = pd.read_csv(r'batteries.csv')

print("The dataset consists of {} samples".format(len(numerical_data)))
print("The numerical data loaded consists of following columns:\n{}".format(
    "\n".join(["\tColumn {}: {}".format(i, x) for i, x in enumerate(numerical_data.columns)])))
print("An example sample:\n{}".format(
    "\n".join(["\tFeature {}: {}={}".format(i, x, v) for i, (x, v) in 
               enumerate(zip(numerical_data.columns, numerical_data.loc[0, :])) if not x == 'Crystal System'])))
print('\tClass (Family): {}'.format(numerical_data.loc[0, 'Crystal System']))

# We split the dataset into features and the target class, using the 'Family' column as the target:
X_nd = numerical_data.loc[:, numerical_data.columns!='Crystal System'].values
y_nd = numerical_data['Crystal System'].values

# We further split the dataset into a training and testing set
X_nd_train, X_nd_test, y_nd_train, y_nd_test = train_test_split(X_nd, y_nd, test_size = 0.2, random_state = 0)

print("The training set consists of {} samples and {} associated ground truth classes".format(len(X_nd_train),
                                                                                             len(y_nd_train)))
print("The test set consists of {} samples and {} associated ground truth classes".format(len(X_nd_test),
                                                                                             len(y_nd_test)))

The dataset consists of 237 samples
The numerical data loaded consists of following columns:
	Column 0: Formation Energy
	Column 1: E Above Hull
	Column 2: Band Gap
	Column 3: Nsites
	Column 4: Density
	Column 5: Volume
	Column 6: Crystal System
An example sample:
	Feature 0: Formation Energy=-2.555
	Feature 1: E Above Hull=0.069
	Feature 2: Band Gap=1.854
	Feature 3: Nsites=15
	Feature 4: Density=2.925
	Feature 5: Volume=179.798
	Class (Family): triclinic
The training set consists of 189 samples and 189 associated ground truth classes
The test set consists of 48 samples and 48 associated ground truth classes


### Categorical data

The second dataset for this assessment contains a mix of numerical and categorical features, with the goal of predicting the forest cover type. The data used in this assessment is adapted from a full dataset provided [here](https://archive.ics.uci.edu/ml/datasets/Covertype) (Jock A. Blackard and Colorado State University).

Each sample corresponds to a 30x30 meter cell, and consists of cartographic variables only (i.e. it does not rely on any remotely sensed imaging data), derived from from US Geological Survey (USGS) and USFS data. Each sample consists of the following features (categorical features can be distinguished by being encoded as a string rather tha a numer):

| Feature Name      | Value | Description |
| :---------------- | :----- | ----------- |
| `Elevation`       | `float`: kilometers | Elevation |
| `Aspect`   | `int`: degrees azimut | Aspect in degreez azimuth |
| `Slope`      | `int`: degrees | Slope in degrees |
| `Horizontal_Distance_To_Hydrology`   | `float`: kilometers | Horiz. distance to nearest surface water feature        |
| `Vertical_Distance_To_Hydrology`   | `float`: kilometers | Vert. distance to nearest surface water feature        |
| `Horizontal_Distance_To_Roadways`     | `float`: kilometers | Hor. distance to nearest roadway       |
| `Hillshade_9am`   |`int`: $0 \dots 255 $|  Hillshade index at 9am, summer solstice        |
| `Hillshade_Noon`      |`int`: $0 \dots 255 $| Hillshade index at noon, summer solstice       |
| `Hillshade_3pm`   |`int`: $0 \dots 255 $| Hillshade index at 3pm, summer solstice        |
| `Horizontal_Distance_To_Fire_Points`  | `float`: kilometers | Horiz. distance to nearest wildfire ignition point        |
| `Wilderness_Area`   | `string`: name | Wilderness area name (4 different values)       |
| `Soil_Type`   |`string`: code| Soil type code (40 different values)       |

The goal of this dataset is to predict the forrest cover type for each 30x30 meter cell. A class, corresponding to the forest cover type, is associated to each sample:

| Class      | Value | Description |
| :---------------- | :----- | ----------- |
| `Cover_Type`       | `string`: name | Forest cover type designation (7 different values). |


**Note:** Tackling the categorical dataset is a **stretch task** for this assessment. It will be used in the development and testing of the [Decision Tree](#Decision-Trees) classifier, as well as for their [evaluation](#Task-4.1---Evaluation). Tackling this data will **require you to implement a** `DecisionTree` **from scratch**, as `sklearn` **does not provide this functionality**.

The following cell loads the dataset, and splits the samples randomly, using  $80\%$ of samples for training and $20\%$ for testing:

In [3]:
categorical_data = pd.read_csv(r'forestcover.csv')

print("The dataset consists of {} samples".format(len(categorical_data)))
print("The numerical data loaded consists of following columns:\n{}".format(
    "\n".join(["\tColumn {}: {}".format(i, x) for i, x in enumerate(categorical_data.columns)])))
print("An example sample:\n{}".format(
    "\n".join(["\tFeature {}: {}={}".format(i, x, v) for i, (x, v) in 
               enumerate(zip(categorical_data.columns, categorical_data.loc[0, :])) if not x == 'Cover_Type'])))
print('\tClass (Cover_Type): {}'.format(categorical_data.loc[0, 'Cover_Type']))

# We split the dataset into features and the target class, using the 'Cover_Type' column as the target:
X_cd = categorical_data.loc[:, categorical_data.columns!='Cover_Type'].values
y_cd = categorical_data['Cover_Type'].values


# We further split the dataset into a training and testing set
X_cd_train, X_cd_test, y_cd_train, y_cd_test = train_test_split(X_cd, y_cd, test_size = 0.2, random_state = 0)

print("The training set consists of {} samples and {} associated ground truth classes".format(len(X_cd_train),
                                                                                             len(y_cd_train)))
print("The test set consists of {} samples and {} associated ground truth classes".format(len(X_cd_test),
                                                                                             len(y_cd_test)))

The dataset consists of 58101 samples
The numerical data loaded consists of following columns:
	Column 0: Elevation
	Column 1: Aspect
	Column 2: Slope
	Column 3: Horizontal_Distance_To_Hydrology
	Column 4: Vertical_Distance_To_Hydrology
	Column 5: Horizontal_Distance_To_Roadways
	Column 6: Hillshade_9am
	Column 7: Hillshade_Noon
	Column 8: Hillshade_3pm
	Column 9: Horizontal_Distance_To_Fire_Points
	Column 10: Wilderness_Area
	Column 11: Soil_Type
	Column 12: Cover_Type
An example sample:
	Feature 0: Elevation=3.0
	Feature 1: Aspect=87.0
	Feature 2: Slope=9.0
	Feature 3: Horizontal_Distance_To_Hydrology=0.3
	Feature 4: Vertical_Distance_To_Hydrology=0.0
	Feature 5: Horizontal_Distance_To_Roadways=3.7
	Feature 6: Hillshade_9am=233.0
	Feature 7: Hillshade_Noon=226.0
	Feature 8: Hillshade_3pm=125.0
	Feature 9: Horizontal_Distance_To_Fire_Points=2.2
	Feature 10: Wilderness_Area=Rawah
	Feature 11: Soil_Type=ELU-7745
	Class (Cover_Type): Lodgepole_Pine
The training set consists of 46480 samp

### Task 1 - Dataset statistics
#### 10% marks

To handle the data for the classification tasks, implement functions that will assist in querying some basic information about the dataset you are handling.

Given a set of M samples $\mathbf{x} = \{\mathbf{x}_i\} \forall i=1..M$ with n features each $\mathbf{x}_i = \{x_{1,i}, \dots, x_{n,i}\}$, and the corresponding ground truth labels $\mathbf{y} = \{y_i\} \forall i=1..M$, implement two functions:

- `class_probabilities(y)` : calcualtes the class probabilities from the list of ground truth labels

    *Inputs:*<br>
    `y`: The list of ground truth labels for a dataset<br>
    
    *Returns:*<br>
    A dictionary of class labels and their probabilities (i.e. the frequency of that class in the dataset)

    *Example* :
    ```python
    y = [1, 0, 0, 0, 1, 2, 1, 1, 2, 1]
    print(class_probabilities(y))
    ```
    *Example output*:<br>
    `{0: 0.3, 1: 0.5, 2: 0.2}`
    
    *Note:* Python dictionaries are unordered, so running this example might result in a different order of keys in the output, for example: `{2: 0.2, 0: 0.3, 1: 0.5}`<br><br>
   
- `most_common_class` : finds the most common class from the list of ground truth class labels.

    *Inputs:*<br>
    `y`: the list of ground truth labels for a dataset<br>
    
    *Returns:*<br>
    the label of the most common class represented in the dataset.
    
    *Example*:
    ```python
    y = [1, 0, 0, 0, 1, 2, 1, 1, 2, 1]
    print(most_common_class(y))
    ```
    *Example output*:<br>
    `1`

#### SOLUTION CELLS:

In [4]:
def class_probabilities(y): 
    classDict = collections.defaultdict(float) #establishes an empty dictionary that will only take floats 
    for i in range(len(y)): 
        if y[i] in classDict: 
            classDict[y[i]] += 1.0 #increases the value of the key (which is the class) 
        else: 
            classDict[y[i]] = 1.0 #makes a new key for a newly found class 
    totalClasses = sum(classDict.values()) #sum together the values 
    for key in classDict: 
        classDict[key] /= totalClasses #gets the probability of each class which is number of instances/total number
    return classDict


In [5]:
def most_common_class(y): 
    classDict = collections.defaultdict(int) #establishes an empty dictionary that will only take ints 
    for i in range(len(y)): 
        if y[i] in classDict: 
            classDict[y[i]] += 1 #increases the value of the key (which is the class)
        else: 
            classDict[y[i]] = 1 #makes a new key for a newly found class 
    sortedClass = sorted(classDict.items(), key = lambda x:x[1], reverse = True) #sorts in ascending order the dictionary
    return sortedClass[0][0] #returns the value seen the most (mode). If there are two modes, returns the first


#### TESTING CELLS:

In [6]:
unq, cnt = np.unique(y_nd, return_counts=True)
print("All unique class labels and counts from numerical-only dataset: {}".format(
    ' '.join(['{}: {}'.format(v,c) for v, c in zip(unq, cnt)])))
print("All classes and their probabilities:\n{}".format(
    "\n".join(["\t{}: {}".format(c, p) for c, p in class_probabilities(y_nd).items()])))
print("Most common class in the numerical-only dataset: {}".format(most_common_class(y_nd)))
print("\n")

unq, cnt = np.unique(y_nd_train, return_counts=True)
print("All unique class labels for the numerical-only training set: {}".format(
    ' '.join(['{}: {}'.format(v,c) for v, c in zip(unq, cnt)])))
print("All classes and their probabilities (num-only training set):\n{}".format(
    "\n".join(["\t{}: {}".format(c, p) for c, p in class_probabilities(y_nd_train).items()])))
print("Most common class in the num-only training set: {}".format(most_common_class(y_nd_train)))
print("\n")

unq, cnt = np.unique(y_nd_test, return_counts=True)
print("All unique class labels for the numerical-only testing set: {}".format(
    ' '.join(['{}: {}'.format(v,c) for v, c in zip(unq, cnt)])))
print("All classes and their probabilities (num-only tests set):\n{}".format(
    "\n".join(["\t{}: {}".format(c, p) for c, p in class_probabilities(y_nd_test).items()])))
print("Most common class in the num-only test set: {}".format(most_common_class(y_nd_test)))
print("\n")

y_test = [1, 0, 0, 0, 1, 2, 1, 1, 2, 1]
print("All class labels for a toy set: {}".format(y_test))
print("All classses and their probabilities for the toy set:\n{}".format(
    "\n".join(["\t{}: {}".format(c, p) for c, p in class_probabilities(y_test).items()])))
print("Most common class in the toy set: {}".format(most_common_class(y_test)))
print("\n")

print("All classes and their probabilities from the categorical dataset:\n{}".format(
    "\n".join(["\t{}: {:.3f}".format(c, p) for c, p in class_probabilities(y_cd).items()])))
print("Most common class in the categorical dataset: {}".format(most_common_class(y_cd)))

All unique class labels and counts from numerical-only dataset: monoclinic: 92 orthorhombic: 89 triclinic: 56
All classes and their probabilities:
	triclinic: 0.23628691983122363
	monoclinic: 0.3881856540084388
	orthorhombic: 0.3755274261603376
Most common class in the numerical-only dataset: monoclinic


All unique class labels for the numerical-only training set: monoclinic: 76 orthorhombic: 69 triclinic: 44
All classes and their probabilities (num-only training set):
	triclinic: 0.2328042328042328
	monoclinic: 0.4021164021164021
	orthorhombic: 0.36507936507936506
Most common class in the num-only training set: monoclinic


All unique class labels for the numerical-only testing set: monoclinic: 16 orthorhombic: 20 triclinic: 12
All classes and their probabilities (num-only tests set):
	triclinic: 0.25
	monoclinic: 0.3333333333333333
	orthorhombic: 0.4166666666666667
Most common class in the num-only test set: orthorhombic


All class labels for a toy set: [1, 0, 0, 0, 1, 2, 1, 1, 2, 

## k Nearest Neighbours

For the first part of the assessment, you are required to implement **k Nearest Neigbours**. A key concept for kNN (and many different machine learning algorithms) is that of **distance**. 

### Task 2.1 - Euclidean distance
#### 10% marks

One of the most commonly used distance metrics is the Euclidean distance. The Euclidean distance between two points in the Euclidean space (also known as $L_2$ norm) is defined as the length of the line segment between those two points. For example, if we have two points in 2D space (i.e. two samples consisting of two features), $\mathbf{x}_0 = \{x_{0,0}, x_{0,1}\}$ and $\mathbf{x}_1 = \{x_{1,0}, x_{1,1}\}$, we can calculate the distance as:

\begin{equation*}
d_{\texttt{Euclidean}} = \sqrt{(x_{0,0} - x_{1,0})^2 + (x_{0,1} - x_{1,1})^2}
\end{equation*}

In other words, we need to calculate the difference between the first features of the two samples, the difference between the second features of the two samples, then square each of the differences, sum those squares, and calculate the square root of the resulting sum.

In general, when we are dealing with points in nD space (corresponding to samples with n features) of the form $\mathbf{x}_i = \{x_{0,i}, x_{1,i}, \dots, x_{n,i}\}$, the procedure is very similar. To calculade the Euclidean disatnce, you need to calculate the difference between the corresponding features of two samples, square each of the differences, and calculate the square root of the resulting sum:

\begin{equation*}
d_{\texttt{Euclidean}} = \sqrt{\sum_i{(x_{0,i} - x_{1,i})^2}}
\end{equation*}

Implement the Euclidean distance as a Python function:
- `euclidean_distance`: calculates the Euclidean distance between two n-dimensional samples

    *Inputs:*<br>
    `x1`: An n-dimensional sample. This either a list of length $n$, or a `numpy.array` of dimensions `(n,)`<br>
    `x2`: An n-dimensional sample. This either a list of length $n$, or a `numpy.array` of dimensions `(n,)`<br>

    *Returns*<br>
    The Euclidean distance between `x1` and `x2`.<br>
    
    *Example:*

    ```python
    x1 = [7, -1]
    x2 = [4, 3]
    print(euclidean_distance(x1, x2)
    ```
    *Example output:*<br>
    `5.0`

    

#### SOLUTION CELL:

In [7]:
def euclidean_distance(x1, x2):
    differenceSquared = 0 #will have a sum of squared values 
    for i in range(len(x1)): #goes through all the values of the array as euclidian distance works through n dimensions  
        differenceSquared += (x1[i] - x2[i]) ** 2 #works out the difference between the two values at dimension i and squares them
    differenceSquared = math.sqrt(differenceSquared) #square roots the total for euclidian distance
    return differenceSquared


#### TESTING CELL:

In [8]:
a = [7, -1]
b = [4, 3]
print("Euclidean distance between {} ({}D) and {} ({}D) is {}".format(
    a, len(a), b, len(b), euclidean_distance(a,b)))
a = [1, 2]
b = [3, 4]
print("Euclidean distance between {} ({}D) and {} ({}D) is {:.3f}".format(
    a, len(a), b, len(b), euclidean_distance(a,b)))
a = [4.2, 3.6, 1.0]
b = [5.7, 2.8, -1.5]
print("Euclidean distance between {} ({}D) and {} ({}D) is {:.3f}".format(
    a, len(a), b, len(b), euclidean_distance(a,b)))
a = np.random.rand(20)
b = np.random.rand(20)
print("Euclidean distance between {} ({}D) and {} ({}D) is {:.3f}".format(
    a, len(a), b, len(b), euclidean_distance(a,b)))

Euclidean distance between [7, -1] (2D) and [4, 3] (2D) is 5.0
Euclidean distance between [1, 2] (2D) and [3, 4] (2D) is 2.828
Euclidean distance between [4.2, 3.6, 1.0] (3D) and [5.7, 2.8, -1.5] (3D) is 3.023
Euclidean distance between [0.20488259 0.60840624 0.86450847 0.81396427 0.26677942 0.28907039
 0.5474658  0.51307861 0.31265578 0.60960883 0.7505621  0.68713959
 0.82353012 0.58371389 0.95004835 0.15862421 0.94601071 0.59987245
 0.82913879 0.51537348] (20D) and [0.19027926 0.9803218  0.05037757 0.44190665 0.71379359 0.65414937
 0.27825911 0.164024   0.62912732 0.90166808 0.50827217 0.86584792
 0.07605024 0.80767127 0.63098106 0.3830714  0.72020191 0.44455909
 0.04767279 0.48849142] (20D) is 1.786


### Task 2.2 - kNN classifier
#### 20% marks

K nearest neighbours classifier will predict the class of a sample based on the class label of its $k$ nearest neigbours, according to a given distance metric. Implement `kNN` classifier which works on samples with numerical features, and relies on the Euclidean distance, as a Python class implementing the following member functions:

- `__init__(self, k = 5, distance = euclidean_distance)`: class constructor

    *Inputs:*<br>
    `k` : The number of neigbours to be considered by the classification model <br>
    `distance`: Tunction which is used to calculate the distance between samples (you will use the Euclidean distance implemented in the previous task)<br><br>
    
- `fit(self, X, y)`: member function used to train the classifier (fit the model to the data)

    *Inputs:*<br>
    `X`: The dataset samples. This is either a list of $M$ lists of length $n$, or a `numpy.array` of dimensions `(M, n)`<br>
    `y`: The dataset labels. This is either a list of length $M$, or a `numpy.array` of dimensions `(M,)`<br><br>
    
    *Returns:*<br>
    The trained classifier model.<br><br>
    
- `predict(self, X)`: member function used to predict the classes of samples `X`

    *Inputs:*<br>
    `X`: The samples for which to predict a class. If this is a single sample, then `X` is either a list of length $n$, or a `numpy.array` of dimensions `(n,)`. If this is a set of samples, then `X` is either a list of $M$ lists of length $n$, or a `numpy.array` of dimensions `(M, N)`<br>
    
    *Returns:*<br>
    Predicted class for the samples of X. If `X` was a single sample, then the output is a single class label. If `X` was a set of samples, then the output is a `numpy.array` of dimensions `(n,`)<br>
    
    *Note:*<br>
    The provided code outline already handles the case where `X` is a set of samples (by running on each sample separately). You need write the main logic of this function, i.e. calculating the prediction for a single sample.<br><br>

#### SOLUTION CELL:

In [9]:
class knn:
    def __init__(self, k = 5, distance = euclidean_distance):
        
        self.k = k
        self.distance = distance #constructor to set values of the class
       
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y #function to fit the training data 

        return self # this should probably be the last line in your fit function
            
    def predict(self, X):
        
        # the fololwing block of code handles lists/arrays containing multiple samples
        X = np.array(X)
        if len(X.shape) >= 2:
            return np.array([self.predict(x) for x in X])
        
        # in the following section, you can assume that X is a single n-dimensional sample:
        self.X = X #test dataset
        distances = {} #dictionary that will contain each index and the distance corresponding to that index
        neighbours = []
        for i in range(len(self.X_train)):
            distances[i] = self.distance(self.X_train[i], X) #uses the euclidian distance function to check distance 
            #between new sample and sample at point i
        sortDist = sorted(distances.items(), key = lambda x:x[1]) #sorts the distances from smallest to largest
        for i in range(self.k):
            neighbours.append(sortDist[i][0]) #gets the k number of shortest distances (which are therefore neighbours)
        classes = []
        for i in range(len(neighbours)):
            classes.append(self.y_train[neighbours[i]]) #neighbours[i] will get the index of a nearest neighbour
            #the class of that nearest neighbour is then appended
        return most_common_class(classes) #finds the common class from the nearest neighbours and assigns sample to this class
        

#### TESTING CELL:

In [10]:
# Initialise the classifier with k=3 (using 9 neigbours)
classifier = knn(k=9)
# Fit the classifier to the training set
classifier.fit(X_nd_train, y_nd_train)

# Predict the classification for some samples to the testing set
for (sample, prediction, truth) in zip(X_nd_test[3:10], classifier.predict(X_nd_test[3:10]), y_nd_test[3:10]):
    # Print the sample, predicted class and ground truth
    print("Sample: {}\n\tPredicted class: {}\n\tTrue class: {}".format(sample, prediction, truth))

Sample: [-2.45500e+00  7.00000e-02  2.34400e+00  2.60000e+01  3.21300e+00
  3.09993e+02]
	Predicted class: triclinic
	True class: orthorhombic
Sample: [-2.35200e+00  8.30000e-02  1.36700e+00  2.80000e+01  2.93500e+00
  3.57496e+02]
	Predicted class: monoclinic
	True class: orthorhombic
Sample: [-2.55300e+00  7.30000e-02  2.64900e+00  3.20000e+01  2.87700e+00
  3.73622e+02]
	Predicted class: monoclinic
	True class: monoclinic
Sample: [-2.43900e+00  5.70000e-02  2.30000e-01  1.50000e+01  3.02400e+00
  1.77312e+02]
	Predicted class: triclinic
	True class: triclinic
Sample: [-2.88700e+00  4.00000e-02  3.14400e+00  5.20000e+01  2.69000e+00
  6.79101e+02]
	Predicted class: orthorhombic
	True class: monoclinic
Sample: [-2.56400e+00  5.80000e-02  2.73000e+00  2.80000e+01  2.85600e+00
  3.60121e+02]
	Predicted class: monoclinic
	True class: orthorhombic
Sample: [-2.91100e+00  6.40000e-02  3.07900e+00  3.40000e+01  2.63300e+00
  4.31422e+02]
	Predicted class: triclinic
	True class: triclinic


## Decision Trees

For the second part of the assignment, you are expected to implement a decision tree classifier. A decision tree models the data as a series of decisions based on the values of different features of the sample. The predictions are made in an interpretable fashion, where the decision is made by considering different sample features, and their values, until the new sample is grouped with similar samples presented during training.

### Task 3.1 - Entropy and Information Gain
#### 10% marks

Important concepts in decision trees are **entropy** and **information gain**. **Entropy** is a measure of the variance in the data, of how "unpredictable" the dataset is:

- If a dataset consists only of samples with the same label (e.g. $\{\blacksquare,\blacksquare,\blacksquare,\blacksquare,\blacksquare,\blacksquare\}$), the entropy will be zero (the data is predictable).
- If a dataset consists of an equal amount of samples from two classes, (e.g. $\{\blacksquare,\triangle,\triangle,\blacksquare,\triangle,\blacksquare\}$), the entropy will be one (the data is unpredictable).
- If the class distribution in the dataset lies somewhere between these two extremes, the entropy will be between one and zero.
- Entropy also increases as the number of different classes increases.

Given $c$ classes, and their associated probabilities (frequencies) in the datasets, $p_i \forall i \in 0..c$, the entropy is calculated as:

\begin{equation*}
E = -\sum_i p_i log_2(p_i).
\end{equation*}

Implement a function `entropy` which calculates the entropy of a set of class labels from the list of their associated probabilities:

- `entropy(p_per_class)`: calculates the entropy of a set of class probabilities

    *Inputs:*<br>
    `p_per_class`: A list of $c$ probabilities, one for each class in the dataset. This either a list of length $c$, or a `numpy.array` of dimensions `(c,)`<br>

    *Returns:*<br>
    The entropy of a set with class probabilities given in `p_per_class`.<br>
    
    *Note:*<br>
    You do not need to check if the list of probabilities given is correct; i.e. you may assume that all the probabilities in the list are strictly greater than zero, and that the sum of all the class probabilities equals one.
    
    *Example:*

    ```python
    probabilities = [0.5, 0.5]
    print(entropy(probabilities))
    ```
    *Example output:*<br>
    `1.0`

#### SOLUTION CELL:

In [11]:
def entropy(p_per_class):
    if len(p_per_class) == 1: #if there is only one sample in the list then calculate the entropy  for one sample
        return -(p_per_class[0] * math.log2(p_per_class[0])) #entropy formula
    elif len(p_per_class) > 2: #if the list is greater than 2 then it needs to be split 
        midpoint = int(len(p_per_class) / 2) #find midpoint of the list
        return entropy(p_per_class[0:midpoint]) + entropy(p_per_class[midpoint:len(p_per_class)]) #recursive
        #above statement will work out the entropy for the left hand side list and add it to the entropy of the right hand side list
    else:
        positive = p_per_class[0]
        negative = p_per_class[1] #works out entropy for two values
        return -(positive * math.log2(positive) + (negative * math.log2(negative))) #formula negative sum of two entropies
    

#### TESTING CELL:

In [12]:
print("Entropy of a set with only one class: {}".format(entropy([1.0])))
print("Entropy of a set with two equally probable classes: {}".format(entropy([0.5, 0.5])))
print("Entropy of a set with four equally probable clasess: {}".format(entropy([0.25, 0.25, 0.25, 0.25])))
print("Entropy of a set with four classes with different probabilities: {:.3f}".format(
    entropy([0.2, 0.3, 0.4, 0.1])))
print("Entropy of the numerical (batteries) dataset: {:.3f}".format(
    entropy(list(class_probabilities(y_nd).values()))))
print("Entropy of the categorical (forest cover) dataset: {:.3f}".format(
    entropy(list(class_probabilities(y_cd).values()))))

Entropy of a set with only one class: -0.0
Entropy of a set with two equally probable classes: 1.0
Entropy of a set with four equally probable clasess: 2.0
Entropy of a set with four classes with different probabilities: 1.846
Entropy of the numerical (batteries) dataset: 1.552
Entropy of the categorical (forest cover) dataset: 1.739


The **information gain** describes how much variance was lost by dividing a set into parts, i.e. how much more "predictable" the parts are from the whole.

For example, if we have a dataset $d=\{\blacksquare,\triangle,\triangle,\blacksquare,\triangle,\blacksquare,\triangle,\blacksquare\}$ (which is "maximally unpredictable", so we can calculate $E(d) = 1$), we can:
- split it into $d_1=\{\blacksquare,\blacksquare,\blacksquare,\blacksquare\}$ and $d_2=\{\triangle, \triangle, \triangle, \triangle\}$.

    Both of $d_1$ and $d_2$ are "very predictable" ($E(d_1) = E(d_2) = 0$), both subsets consist of one class only). The resulting information gain is large (we go from unpredictable to predictable, so we gain information):
    
    $I = E(d) - (0.5E(d_1) + 0.5E(d_2) = 1 - (0+0) =1$
    
- split it into $d_1=\{\blacksquare,\triangle,\triangle,\blacksquare\}$ and $d_2=\{\triangle,\blacksquare,\triangle,\blacksquare\}$.

    Both of $d_1$ and $d_2$ are "very unpredictable" ($E(d_1) = E(d_2) = 1$), both subsets have the same amount of samples from each of the two classes). The resulting information gain is small (we start from an unpredictable dataset, and end with two unpredictable subsets, so we do not gain information):

    $I = E(d) - (0.5E(d_1) + 0.5E(d_2) = 1 - (0.5\times1+0.5\times1) =0$
    
<br>
In general, if we have a dataset $d$ containing $|d|$ samples, and we split it into $s$ subsets $d_i \forall i=1..s$, each of size $|d_i|$, the information gain of this split is calculated as:

\begin{equation*}
    I = E(d) - \sum_{i=0}^s \frac{|d_i|}{|d|}E(d_i)
\end{equation*}

### Task 3.2 - Decision Tree classifier
#### 30% marks

In each decision node of a decision tree, this classifier splits the dataset into two or more parts according to a value of a certain feature in the dataset.

Given a set of M samples $\mathbf{x} = \{\mathbf{x}_i\} \forall i=1..M$ with n features each $\mathbf{x}_i = \{x_{1,i}, \dots, x_{n,i}\}$, we could decide to split the dataset at the $j^{\texttt{th}}$ (numerical) feature at value $v$. This would mean splitting the dataset into:
- the subset $\mathbf{x_{<}} = \{\mathbf{x}_i | x_{j,i} < v\}$, which contains all the samples with the $j^{\texttt{th}}$ smaller than $v$ and
- the subset $\mathbf{x_{\geq}} = \{\mathbf{x}_i | x_{j,i} \geq v\}$, which contains all the samples with the $j^{\texttt{th}}$ greater or equal than $v$.

Similarly, we could decide to split a dataset at the $j^{\texttt{th}}$ feature which is categorical. If the $j^{\texttt{th}}$ feature of a sample $\mathbf{x}_i$ can hold values $\{\texttt{child}, \texttt{teen}, \texttt{adult}\}$, the resulting subsets would be:
- the subset $\mathbf{x_{\texttt{child}}} = \{\mathbf{x}_i | x_{j,i} = \texttt{child}\}$, which contains all the samples with the feature $j^{\texttt{th}}$ equal to $\texttt{child}$, 
- the subset $\mathbf{x_{\texttt{teen}}} = \{\mathbf{x}_i | x_{j,i} = \texttt{teen}\}$, which contains all the samples with the feature $j^{\texttt{th}}$ equal to $\texttt{teen}$ and
- the subset $\mathbf{x_{\texttt{adult}}} = \{\mathbf{x}_i | x_{j,i} = \texttt{adult}\}$, which contains all the samples with the feature $j^{\texttt{th}}$ equal to $\texttt{adult}$.

In each node of the decision tree, the dataset is split according to the attribute $j$ which results in **the highest information gain** after the split. For categorical features, there is only one possible split (each category becomes a subset). For numerical features, it is neccessary to check all the possible values of split. For example, if the $j^{\texttt{th}}$ in the dataset has appeared with the values of $\{1, 1.5, 2.7, 3.2, 5\}$, you need to calculate the information gain for splitting the dataset at $v=1.5$ (into $\mathbf{x_{< 1.5}}$ and $\mathbf{x_{\geq 1.5}}$), $v=2.7$, $v=3.2$, $v=5$.

The splitting continues until each node contains only samples of a single class, or a stopping criterion is reached. If a node contains training samples of more than one class, it returns the label of the majority (most probable) class as its decision.

Implement a `DecisionTree` classifier which works on samples with both numerical and categorical features as a Python class implementing the following member functions:

- `__init__(self, max_depth = -1, min_samples_per_node = 1)`: class constructor

    *Inputs:*<br>
    `max_dept` : The maximal depth of the tree (`max_depth == 0` corresponds to a decision tree with only the root node. `max_depth == -1` means no limit to the depth of the tree) <br>
    `min_samples_per_node`: The minimal number of samples contained in a terminal (decision) node. If a split would cause one of the subsets to have less than `min_samples_per_node` samples, the split is not performed. <br><br>
    
- `fit(self, X, y)`: member function used to train the classifier (fit the model to the data)

    *Inputs:*<br>
    `X`: The dataset samples. This is either a list of $M$ lists of length $n$, or a `numpy.array` of dimensions `(M, n)`<br>
    `y`: The dataset labels. This is either a list of length $M$, or a `numpy.array` of dimensions `(M,)`<br>
    
    *Returns:*<br>
    The trained classifier model.<br><br>
    
- `predict(self, X)`: member function used to predict the classes of samples `X`

    *Inputs:*<br>
    `X`: The samples for which to predict a class. If this is a single sample, then `X` is either a list of length $n$, or a `numpy.array` of dimensions `(n,)`. If this is a set of samples, then `X` is either a list of $M$ lists of length $n$, or a `numpy.array` of dimensions `(M, N)`<br>
    
    *Returns:*<br>
    Predicted class for the samples of X. If `X` was a single sample, then the output is a single class label. If `X` was a set of samples, then the output is a `numpy.array` of dimensions `(n,`)<br>
    
    *Note:*<br>
    The provided code outline already handles the case where `X` is a set of samples (by running on each sample separately). You need write the main logic of this function, i.e. calculating the prediction for a single sample.<br><br>
    
*Note:* Your implementation will be tested in two parts. Firstly, it will be tested only on a dataset containing numerical values. Secondly, it will also be tested on a dataset containing a mix of categorical and numerical values.

#### SOLUTION CELL:

In [19]:
#the following classes and functions were created using this tutorial: Assembly AI (2022) How to implement decision trees from scratch with python [video]. Available from https://www.youtube.com/watch?v=NxEHSAfFlK8&t=974s [accessed 13 December 2022]
class Node:
    def __init__(self, feature = None, threshold = None, left = None, right = None, *, value = None): #asterisk means value must be passed by name
        self.feature = feature #feature that was used to divide the node
        self.threshold = threshold #specific threshold that was used to divide the node
        self.left = left #left tree of node
        self.right = right #right tree of node
        self.value = value #value that will be assigned
        #constructor of the class
    def isLeaf(self):
        return self.value is not None #if a value is assigned to a node then it means a leaf node has been created
    
class DecisionTree:
    def __init__(self, max_depth = -1, min_samples_per_node = 1):
        self.max_depth = max_depth; #constructor of the class assigned the depth of the tree and the number of samples for each node
        self.min_samples_per_node = min_samples_per_node
        self.root = None #access of root node
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y #decision tree fitted to the training data
        self.root = self.grow(X, y) #function used to create the decision tree itself, which becomes the root node
        
    def grow(self, X, y, depth = 0):
        samples, features = X.shape #shape will get number of rows and columns, but we are only really interested in columns
        labels = len(np.unique(y)) #gets the number of unique classes
        if (depth == self.max_depth or labels == 1 or samples < self.min_samples_per_node): 
            #a leaf node will be made if the current depth is the max_depth, the number of labels in the node is 1
            #or the number of samples is less than the minimum samples per node
            leaf = most_common_class(y) #finds the class of the leaf node
            #as most common class is returning the first value in a tie this could lead to varying results
            return Node(value=leaf) #creates a leaf node object with the class value assigned
        featuresIndex = range(features) #creates a list of column indexes for splitting
        best_feature, best_threshold = self.best_split(X, y, featuresIndex)
        left, right = self.split(X[:, best_feature], best_threshold)
        left = self.grow(X[left, :], y[left], depth + 1)
        right = self.grow(X[right, :], y[right], depth + 1)
        return Node(best_feature, best_threshold, left, right)
    
    def best_split(self, X, y, features):
        best_gain = -1 
        split_index, split_threshold = None, None #best gain, split_index and split_threshold created and will be assigned to later
        for feature in features: #for each index in the list of indexes
            col = X[:, feature] #get the values in the column the index points towards
            thresholds = np.unique(col) #find what values are unique in the column
            for threshold in thresholds: #for each unique value
                gain = self.infoGain(y, col, threshold) #find information gain for each threshold
                if gain > best_gain: #if the gain found is better than the existing gain, best_gain becomes the gain calculated
                    best_gain = gain
                    split_index = feature #holds the best index for splitting
                    split_threshold = threshold #holds the best threshold for splitting
        return split_index, split_threshold
    
    def infoGain(self, y, col, thres):
        parentProb = class_probabilities(y) #find all probabilities for each class for the whole of y
        parentList = list(parentProb.values()) #change the returned dictionary to a list
        parentEntropy = entropy(parentList) #calculate the entropy for the whole of dataset y
        left, right = self.split(col, thres) #split dataset based on thrsehold value given
        if (len(left) == 0 or len(right) == 0): #if one list is null then return
            return 0
        samples = len(y) #number of total samples
        leftSamples, rightSamples = len(left), len(right) #number of samples in the left and right trees
        probLeft, probRight = class_probabilities(y[left]), class_probabilities(y[right]) #find all probabilities for classes in the left and right trees
        listLeft, listRight = list(probLeft.values()), list(probRight.values()) #convert to lists
        entropyLeft, entropyRight = entropy(listLeft), entropy(listRight) #gets the entropy of both branches
        childEntropy = (leftSamples/samples) * entropyLeft + (rightSamples/samples) * entropyRight #entropy of a child node is
        #number of samples in left branch/total samples * left branch entropy + number of samples in right branch/total samples * right branch entropy
        gain = parentEntropy - childEntropy #information gain is parent entropy subtracted by child entropy
        return gain
    
    def split(self, col, thres):
        if isinstance(col, str): #splits for categorical values
            lefty = np.argwhere(col == thres).flatten() #returns all values where the column is equal to the threshold 
            righty = np.argwhere(col != thres).flatten() #returns all values where the column is not equal to threshold
            #this may be wrong as the split becomes left for one categorical value and right for all other categorical values
        else: #splits for numerical values
            lefty = np.argwhere(col <= thres).flatten() #returns all values where the column is less than or equal to the split threshold
            righty = np.argwhere(col > thres).flatten() #returns all values where the column is greater than the split threshold
        #flatten creates the format as one list as opposed to multiple lists, aargwhere returns all values that satisfy the decision in the paranthesis
        return lefty, righty
    
    def predict(self, X):
        return np.array([self.traverse(x, self.root) for x in X]) #for every value in X, go through the tree
    
    def traverse(self, x, node):
        if node.isLeaf():
            return node.value #if the end of the tree is reached return the value in the tree
        
        if x[node.feature] <= node.threshold: #if the feature is smaller than the threshold value then go across the left tree
            return self.traverse(x, node.left)
        else: #otherwise go on the right tree
            return self.traverse(x, node.right)

#### TESTING CELLS:

The following cell will test if the classifier implemented model can fit to the [numerical dataset](#Numerical-data), and predict solutions for some of the testing samples (this is not model evaluation yet):

In [20]:
# Initialise the classifier, requiring at least 3 samples in each node
classifier = DecisionTree(min_samples_per_node = 3)
# Fit the classifier to the training data
classifier.fit(X_nd_train, y_nd_train)
# Predict the classification for some samples to the testing set
for (sample, prediction, truth) in zip(X_nd_test[3:10], classifier.predict(X_nd_test[3:10]), y_nd_test[3:10]):
    # Print the sample, predicted class and ground truth
    print("Sample: {}\n\tPredicted class: {}\n\tTrue class: {}".format(sample, prediction, truth))

Sample: [-2.45500e+00  7.00000e-02  2.34400e+00  2.60000e+01  3.21300e+00
  3.09993e+02]
	Predicted class: monoclinic
	True class: orthorhombic
Sample: [-2.35200e+00  8.30000e-02  1.36700e+00  2.80000e+01  2.93500e+00
  3.57496e+02]
	Predicted class: monoclinic
	True class: orthorhombic
Sample: [-2.55300e+00  7.30000e-02  2.64900e+00  3.20000e+01  2.87700e+00
  3.73622e+02]
	Predicted class: triclinic
	True class: monoclinic
Sample: [-2.43900e+00  5.70000e-02  2.30000e-01  1.50000e+01  3.02400e+00
  1.77312e+02]
	Predicted class: orthorhombic
	True class: triclinic
Sample: [-2.88700e+00  4.00000e-02  3.14400e+00  5.20000e+01  2.69000e+00
  6.79101e+02]
	Predicted class: monoclinic
	True class: monoclinic
Sample: [-2.56400e+00  5.80000e-02  2.73000e+00  2.80000e+01  2.85600e+00
  3.60121e+02]
	Predicted class: orthorhombic
	True class: orthorhombic
Sample: [-2.91100e+00  6.40000e-02  3.07900e+00  3.40000e+01  2.63300e+00
  4.31422e+02]
	Predicted class: monoclinic
	True class: triclinic

The following cell will check if the dataset is able to fit the model to the [dataset with mixed numerical and categorical values](#Categorical-data), and predict solutions for some of the testing samples (this is not model evaluation yet):

_**Warning:**_ Executing this cell may take a bit of time.

In [21]:
# Initialise the classifier, requiring the decision tree to have no more than 5 levels
classifier = DecisionTree(max_depth = 4)
# Fit the classifier to the training data
classifier.fit(X_cd_train, y_cd_train)

# Predict the classification for some samples to the testing set
for (sample, prediction, truth) in zip(X_cd_test[3:10], classifier.predict(X_cd_test[3:10]), y_cd_test[3:10]):
    # Print the sample, predicted class and ground truth
    print("Sample: {}\n\tPredicted class: {}\n\tTrue class: {}".format(sample, prediction, truth))

Sample: [2.4 48.0 11.0 0.0 0.0 0.4 224.0 215.0 123.0 1.3 'Cache_la_Poudre'
 'ELU-2717']
	Predicted class: Ponderosa_Pine
	True class: Cottonwood/Willow
Sample: [3.3 67.0 16.0 0.1 0.0 1.7 234.0 207.0 100.0 1.9 'Rawah' 'ELU-8772']
	Predicted class: Spruce/Fir
	True class: Spruce/Fir
Sample: [3.3 60.0 15.0 0.1 -0.0 2.4 231.0 207.0 104.0 4.1 'Neota' 'ELU-8703']
	Predicted class: Spruce/Fir
	True class: Spruce/Fir
Sample: [3.2 62.0 10.0 0.4 0.0 2.0 229.0 219.0 122.0 0.9 'Comanche_Peak'
 'ELU-7755']
	Predicted class: Spruce/Fir
	True class: Spruce/Fir
Sample: [2.9 121.0 6.0 0.5 0.1 3.6 230.0 235.0 139.0 3.1 'Rawah' 'ELU-4744']
	Predicted class: Lodgepole_Pine
	True class: Lodgepole_Pine
Sample: [3.2 155.0 15.0 0.2 0.0 0.4 237.0 240.0 129.0 2.3 'Neota' 'ELU-4758']
	Predicted class: Spruce/Fir
	True class: Spruce/Fir
Sample: [3.0 330.0 8.0 0.1 0.0 2.6 201.0 231.0 169.0 6.2 'Rawah' 'ELU-7745']
	Predicted class: Lodgepole_Pine
	True class: Spruce/Fir


## Model evaluation and analysis

Executing all the cells from the previous section correctly only means that your implementation of [kNN classifier](#knn-Classifier) and [Decision Tree classifier](#Decision-Tree-classifier) runs on the provided data. This has still not provided any insight on how well those models represent the data.

In this section, you are instead required to calculate some evaluation metrics, and evaluate the produced models on the whole of the held-out data (test set) defined in the [Dataset](#Dataset) section.

### Task 4.1 - Evaluation
#### 10% marks

You are required to write a function that will calculate different evaluation metrics based on the ground truth labels and the predicted labels. Specifically, write a function:
- `evaluate(y_true, y_pred)`: evaluate classification results

    *Inputs:*<br>
    `y_true` : Ground truth labels for $n$ samples. This is either a list of $n$ elements, or a `numpy.array` of dimensions `(n,)`
    `y_pred` : Predicted labels for $n$ samples. This is either a list of $n$ elements, or a `numpy.array` of dimensions `(n,)`
    
    *Returns:*<br>
    A touple `(a, p, r, k)` consisting of the following values:
    - `a` : overall accuracy of the predictions
    - `p` : a dictionary containing a precision score for each class present in `y_true`
    - `r` : a dictionary containing a recall score for each class present in `y_true`
    - `k` : Cohen's Kappa metric

#### SOLUTION CELL:

In [22]:
from sklearn import metrics

def evaluate(y_true, y_pred):
    precisionDict = {}
    recallDict = {} #two dictionaries to return precision and recall for class
    values = len(np.unique(y_true)) #finds the number of unique labels needed for key values in dictionary
    labels = np.unique(y_true) #finds the unique labels needed for key values in dictionary
    precision, recall, fscore, support = metrics.precision_recall_fscore_support(y_true, y_pred, labels = np.unique(y_true))
    #sklearn function to find precision, recall, fscore and support but we will only use precision and recall
    #support would return the number of occurances in y_true (Sklearn.metrics.precision_recall_fscore_support (undated) scikit. Available from: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html [accessed 17 December 2022].
    #fscore is a weighted harmonic mean of the precision and recall (Sklearn.metrics.precision_recall_fscore_support (undated) scikit. Available from: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html [accessed 17 December 2022].
    #returns a list for each
    for i in range(values):
        precisionDict[labels[i]] = precision[i] #assigns the precision to the corresponding label
        recallDict[labels[i]] = recall[i] #assigns the recall to the corresponding label
    accuracy = metrics.accuracy_score(y_true, y_pred) #sklearn function that calculates accuracy
    cohenKappa = metrics.cohen_kappa_score(y_true, y_pred) #sklearn function that calculates cohen's kappa
    return accuracy,precisionDict,recallDict,cohenKappa
    

#### TESTING CELL:

The following cell evaluates your implementation of [kNN classifier](#Task-2.2---kNN-classifier) and [Decision Tree classifier](#Task-3.2---Decision-Tree-classifier) on the numerical dataset defined in the [Dataset](#Dataset) section:

In [24]:
classifier_knn = knn(k=5)
classifier_knn.fit(X_nd_train, y_nd_train)

classifier_dt = DecisionTree(min_samples_per_node = 3)
classifier_dt.fit(X_nd_train, y_nd_train)

y_knn = classifier_knn.predict(X_nd_test)
y_dt = classifier_dt.predict(X_nd_test)

a_knn, p_knn, r_knn, k_knn = evaluate(y_nd_test, y_knn)
a_dt, p_dt, r_dt, k_dt = evaluate(y_nd_test, y_dt)

print("Evaluating numerical dataset, using kNN:")
print("Accuracy = {:.4f}, Cohen's Kappa = {:.4f}".format(a_knn, k_knn))
for key in p_knn.keys():
    print("\tClass {}: precision = {:.2f}, recall = {:.2f}".format(key, p_knn[key], r_knn[key]))

print("Evaluating numerical dataset, using DecisionTree:")
print("Accuracy = {:.4f}, Cohen's Kappa = {:.4f}".format(a_dt, k_dt))
for key in p_dt.keys():
    print("\tClass {}: precision = {:.2f}, recall = {:.2f}".format(key, p_dt[key], r_dt[key]))

Evaluating numerical dataset, using kNN:
Accuracy = 0.5833, Cohen's Kappa = 0.3651
	Class monoclinic: precision = 0.56, recall = 0.62
	Class orthorhombic: precision = 0.67, recall = 0.60
	Class triclinic: precision = 0.50, recall = 0.50
Evaluating numerical dataset, using DecisionTree:
Accuracy = 0.5417, Cohen's Kappa = 0.2997
	Class monoclinic: precision = 0.43, recall = 0.62
	Class orthorhombic: precision = 0.62, recall = 0.50
	Class triclinic: precision = 0.67, recall = 0.50


The following cell evaluates your implementation of [Decision Tree classifier](#Decision-Tree-classifier) on the [dataset containing a mixture of numerical and categorical data](#Categorical-data):

_**Warning:**_ Executing this cell may take a bit of time.

In [25]:
classifier_categorical = DecisionTree(max_depth = 4)
classifier_categorical.fit(X_cd_train, y_cd_train)

y_cd_pred = classifier_categorical.predict(X_cd_test)
a_cat, p_cat, r_cat, k_cat = evaluate(y_cd_test, y_cd_pred)

print("Evaluating categorical dataset, using DecisionTree:")
print("Accuracy = {:.4f}, Cohen's Kappa = {:.4f}".format(a_cat, k_cat))
for key in p_cat.keys():
    print("\tClass {}: precision = {:.2f}, recall = {:.2f}".format(key, p_cat[key], r_cat[key]))


Evaluating categorical dataset, using DecisionTree:
Accuracy = 0.6987, Cohen's Kappa = 0.5061
	Class Aspen: precision = 0.00, recall = 0.00
	Class Cottonwood/Willow: precision = 0.00, recall = 0.00
	Class Douglas-fir: precision = 0.46, recall = 0.05
	Class Krummholz: precision = 0.68, recall = 0.52
	Class Lodgepole_Pine: precision = 0.74, recall = 0.75
	Class Ponderosa_Pine: precision = 0.64, recall = 0.86
	Class Spruce/Fir: precision = 0.67, recall = 0.71


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Task 4.2 - Analysis
#### 10% marks

Answer the following questions in the space provided below (marked as **YOUR ANSWERS**):
1. Explain the difference between the different performance metrics calculated for [Task 4.1](#Task-4.1---Evaluation). Which additional metrics could you propose to evaluate the performance of the defined models? Which of the calculated metrics would you chose to report, and why, for the analysis of the two [datasets](#Datasets) used in this assessment?

2. In case you were handling a _very large_ amount of data, which of the models do you expect to be larger (i.e. take up more memory)? Which of the models do you expect to train faster? Which of the models do you expect to reach a decision faster? Explain and justify your answers.

3. When handling numerical data, what is the role of normalisation? The numerical data used in this assessment was not normalised. If you were to normalise the data before training these classification models, would either of them change how they represent the data (and potentially return different results)?

4. For evaluating the performance of the classifiers implemented for this assignment, $80\%$ of the data was used for training the models, and $20\%$ for testing. What are the shortcomings of this evaluation strategy, and can you propose and describe a better one?

_Note:_ You may use the cell marked as **EXPERIMENTAL CELL** to write any code that could help you answer the above questions, or in general, to experiment with the code produced for this assessment.

#### EXPERIMENTAL CELL:

#### YOUR ANSWERS:

1. <font color='red'>Answer to question 1

Using kNN for the batteries dataset returned an accuracy of 0.5833, a Cohen's Kappa of 0.3651 and the precision and recall for monoclinic, orthorhombic and triclinic was 0.56 and 0.62, 0.67 and 0.60 and 0.5 and 0.5 respectively.
For the decision tree, the accuracy was 0.5417 and Cohen's Kappa was 0.2997. For monoclinic, orthorhombic and triclinic the precision and recall were 0.43 and 0.62, 0.62 and 0.5 and 0.67 and 0.5 respectively. Therefore, overall we could say that kNN was more accurate than a decision tree, with accuracy being True Positives and True Negatives / Total. Precision is True Positives / Number of Predicted Yeses. On average this was 0.577 (3sf) for kNN and 0.573 (3sf) for a decision tree, meaning kNN is more precise. Recall is True Positives / Actual Number of Yeses which for kNN was an average of 0.573 (3sf) compared to 0.54 for a decision tree, meaning kNN was better at recall. Finally, Cohen's Kappa is defined as "the level of agreement between two raters or judges who each classify items into mutually exclusive categories." (Zach, 2021). Again kNN produces a higher score, this time for Cohen's Kappa, showing that there is a better level of agreement between the model and the true values.

Additional metrics that could have been used are specificity, which evaluates the number of True Negatives / Actual Number of Nos, the false positive rate which is the False Positives / Actual Number of Nos (however this is just 1 - specificity) or the misclassification rate, which is the number of False Positives and False Negatives / Total 
(however this is 1 - accuracy). Overall, the better model was kNN as it outperformed decision trees for each metric. This could be because the batteries dataset had a max_depth of -1, which would mean that there was no stopping condition for depth. This could have led to the tree overfitting the data as the tree would be very deep and so would only be memorising the training data as opposed to working on unseen examples.

Of all the metrics that I would choose to report, I would choose recall. This is because recall can be used to plot an ROC curve, which is a way to determine how good the models are at classifying data. By obtaining both recall and the false positive rate, we could plot an ROC curve by having recall on y and false positive rate on x. It would also be possible to change the values of k in kNN together with the depth in the decision tree to obtain different recall values (and false positive rate values) and then plot these on the ROC curve. We could then work out the area under the graph. The closer the area under the graph is to 1, the better the model is at predicting data (Shaneyfelt, 2019). This could mean that, on average, the decision tree could be a better classifier overall if it's area under the curve that was closer to 1 and thus is better at predicting data.  

References:

Terry Shaneyfelt (2019) How to interpret ROC curves [video]. Available from https://www.youtube.com/watch?v=egTNM8NSa2k [accessed 17 December 2022]. 
Zach (2021) Cohen's kappa statistic: Definition & Example, Statology. Available from https://www.statology.org/cohens-kappa-statistic/ [accessed 17 December 2022]. </font> 

2. <font color='red'>Answer to question 2

KNN is a non-parametric model. This means that no assumptions are made about the data and that the number of parameters is equal to the number of data points. As a result it does not scale well. As the number of data samples increases so the model will need to look at every single data point in order to obtain the k best samples. As the sample size increases, the model will take a larger amount of time to analyize all data points. Equally, decision trees are also non-parametric and, due to being created recursively they will have a "memory complexity of O(N)" (StackOverflow, 2017). However, kNN does have a "space complexity of O(Nd)" (Kumar, 2021) where "d is dimensionality" (Kumar, 2021) which simplifies to O(N). Since there is a scalar involved with kNN, kNN will likely use more memory for very large datasets than the recursive decision tree. 
    
The model that trains fastest is kNN. This is because it has a "time complexity of O(kNd)" (Kumar, 2021) where k is number of neighbours and d is "dimensionality of the data" (Kumar, 2021). Decision trees have a "time complexity of O(dNlogN)" (Kumar, 2021) where d is "the dimensionality of the data" (Kumar, 2021). Simplified this means that kNN is O(N) and decision trees are O(NlogN) which in turn means that kNN has the smaller time complexity and therefore trains faster. During runtime, the time complexity for decision trees will be "O(d) where d is the depth of the tree" (Kumar, 2021). This would essentially end up being O(N) complexity. Equally time complexity at runtime for kNN is "O(nd+kn) runtime or O(ndk) runtime" (StackExchange, 2016) which would again simplify to O(N). The model that reaches a decision faster will be determined by whether the number of samples for kNN is less than the depth of the tree in the decision tree. If kNN has fewer samples (and the values of k and d are low) a decision will be reached quicker to predict a class. If not, the decision tree will predict the class quicker. However, as it's unlikely that the depth of the tree would be greater than the number of samples in kNN, it's more likely that the decision tree will be quicker.

References:

Kumar, P. (2021) Time complexity of ML Models, Medium. Analytics Vidhya. Available from https://medium.com/analytics-vidhya/time-complexity-of-ml-models-4ec39fad2770 [accessed 17 December 2022].
StackExchange (2016) k-NN computational complexity Stack Exchange. Available from https://stats.stackexchange.com/questions/219655/k-nn-computational-complexity [accessed 19 December 2022].
StackOverflow (2017) Space complexity of recursive function Stack Overflow. Available from https://stackoverflow.com/questions/43298938/space-complexity-of-recursive-function [accessed 17 December 2022].
 </font> 

3. <font color='red'>Answer to question 3

The role of normalisation in numerical data is "the process of casting the data to the specific range, like between 0 and 1" (Ali et all, 2014, p1) and is specifically done "when there are big differences in the ranges of different features" (Ali et all, 2014, p1). The way to normalize data for machine learning is "xnorm = (xi – xmin) / (xmax – xmin)" (Zach, 2021) and is used when there are no outliers in the data (Ali et all, 2014, p1). There is another method known as Z-Score Standardization which assumes a normal distribution of data and is worked out by "(x[i]-mean of x)/standard deviation of x" (Ali et all, 2014, p5). Data should be normalized as "normalization ensures that no one variable may influence the performance of the model in a particular way simply due to the fact that it has larger numbers." (Croydon Early Learning, 2022). If, therefore, there were certain values in the dataset that were higher than other values, it could decrease the model's accuracy, so by normalizing the data any discrepancies are removed while making sure the relationship of the data is still the same. This may create better models and therefore more accurate results.

References:

Ali, P.J.M., Faraj, R.H., Koya, E., Ali, P.J.M. and Faraj, R.H., 2014. Data normalization and standardization: a technical report. Mach Learn Tech Rep, 1(1), pp.1-6. [accessed 17 December 2022].
Kiligann, A. (2022) Why do we normalize data in machine learning?, Croydon Early Learning. Available from https://croydonearlylearning.com.au/education/why-do-we-normalize-data-in-machine-learning.html [accessed 17 December 2022]. 
Zach (2021) How to normalize data in python, Statology. Available from https://www.statology.org/normalize-data-in-python/ [accessed 17 December 2022]. </font> 

4. <font color='red'>Answer to question 4

There are various shortcomings in using a training test split for training an algorithm. One main disadvantage is that while a simple train-test split is cheap, it is unreliable for future performance, especially when the dataset used is very small. "There will also not be enough data in the test set to effectively evaluate the model performance. The estimated performance could be overly optimistic (good) or overly pessimistic (bad)." (Brownlee, 2020), meaning that our model may be more unpredictable when testing the data and, as a result, may not be fully accurate. 
    
A better way to test the data would be to use Cross Validation (CV). There are two types that can be used, Leave One Out Cross Validation or k-fold Cross Validation. K-fold separates the data into k number of train-test splits. This means that for each iteration different values in the dataset will be used for each test set. Therefore we obtain a higher proportion of testing data overall, as more samples are used in testing, meaning a more reliable future performance. LOOCV takes one sample from the dataset for testing, trains the rest of the data and then repeats this for all the samples in the dataset. As every sample is being tested once in LOOCV it means we obtain the highest amount of testing data possible, which thus makes it the most reliable for future performance. A drawback in using CV for k-fold, is that it can get computationally expensive if k is large. In using LOOCV, every sample is being used for CV and so large datasets will also be computationally expensive. 
    
We could also, if we were to use a simple train-test split, introduce a validation set of say 10% of the data. This could be used to help prevent overfitting our model and is done during training to see how well the model is perfoming on unseen data. As the data is trained, both the training and validation error will decrease. However, if the validation error starts to increase, then the model starts to be overfitted and the programmer can choose to stop training the model. This is called early stopping.

References:

Brownlee, J. (2020) Train-test split for Evaluating Machine Learning Algorithms, MachineLearningMastery.com. Available from https://machinelearningmastery.com/train-test-split-for-evaluating-machine-learning-algorithms/ [accessed 17 December 2022]. </font> 